In [1]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install pandas

In [30]:
import glob
import pandas as pd
import os

import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets
import datetime

In [3]:
path = r'/Users/paulbrown/Documents/Python/raw_data_files/'

In [4]:
#be careful, the order in which you have your csv files saved matters
all_csv = glob.glob(path + "*.csv")

In [5]:
df_list = []

for filename in all_csv:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['Year'] = os.path.basename(filename)
    #add the year to the file being appended (filename is the year)
    df['Year'] = df['Year'].str[:4]
    df_list.append(df)

### Grab the list of dataframes and sort through the mismatching column headers

In [6]:
#year 2015 and 2016 are similar with the exception of unnecessary columns

df_1516 = pd.concat(df_list[2:4], sort=False, axis=0)
df_1516 = df_1516.drop(['Standard Error','Lower Confidence Interval',
                        'Upper Confidence Interval','Happiness Rank'], axis=1)

In [7]:
#year 2018 and 2019 are similar with the exception Overall Rank. Also, change column names to match the other years

df_1819 = pd.concat(df_list[0:2], sort=False, axis=0).rename(columns={'Country or region':'Country',
                                                                      'Score':'Happiness Score',
                                                                     'GDP per capita':'Economy (GDP per Capita)',
                                                                     'Freedom to make life choices':'Freedom',
                                                                     'Perceptions of corruption':'Trust (Government Corruption)',
                                                                     'Healthy life expectancy':'Health (Life Expectancy)'})
df_1819 = df_1819.drop(['Overall rank'], axis=1)

In [8]:
#year 2017 is disgusting, you basically have to change all the column headers and remove useless whisker stats

df_17 = df_list[4]
df_17 = df_17.drop(['Happiness.Rank','Whisker.high','Whisker.low'], axis=1)

column_map = {'Happiness.Score':'Happiness Score', 
 'Economy..GDP.per.Capita.':'Economy (GDP per Capita)', 
 'Health..Life.Expectancy.':'Health (Life Expectancy)',
 'Trust..Government.Corruption.':'Trust (Government Corruption)',
 'Dystopia.Residual':'Dystopia Residual'
}
df_17 = df_17.rename(columns=column_map)

In [9]:
#bring 15, 16, 17 together and change the column name 'Family to Social support'

df_151617 = pd.concat([df_1516, df_17], sort=False)
df_151617 = df_151617.rename(columns={'Family':'Social support'})

In [10]:
master_df = pd.concat([df_1819, df_151617], sort=False, ignore_index=True)
master_df = master_df.drop(['Region','Dystopia Residual'], axis=1)

In [11]:
#Overall Rank
master_df['Overall Rank'] = master_df['Happiness Score'].rank(ascending=False, method='max').astype(int)

#Rank within Year
master_df['Yearly Rank'] = master_df.sort_values(by=['Year','Happiness Score'], ascending=False)['Happiness Score'].index + 1

### Now you have a dataframe of happiness data, Hurray!
What are you going to do with it? Maybe we could add:
* Region Classification (https://meta.wikimedia.org/wiki/List_of_countries_by_regional_classification)

* Income Inequality (https://en.wikipedia.org/wiki/List_of_countries_by_income_equality)

* Household Debt (https://en.wikipedia.org/wiki/List_of_countries_by_household_debt)

* Economic Freedom (https://en.wikipedia.org/wiki/List_of_countries_by_economic_freedom)

* Unemployment Rate (https://en.wikipedia.org/wiki/List_of_countries_by_unemployment_rate)

* Development Aid Donotions (https://en.wikipedia.org/wiki/List_of_development_aid_country_donors)

* Suicide Rate (https://en.wikipedia.org/wiki/List_of_countries_by_suicide_rate)

* Life Expectancy (https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy)

* Intentional Homicide Rate (https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate)

* Dependency Ratio (https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate)

* Education Index (https://en.wikipedia.org/wiki/Education_Index)

* Number of Internet Users (https://en.wikipedia.org/wiki/List_of_countries_by_number_of_Internet_users)

In [12]:
master_df.sort_values(by=['Year','Happiness Score'], ascending=False).head(5)

,Country,Happiness Score,Economy (GDP per Capita),Social support,Health (Life Expectancy),Freedom,Generosity,Trust (Government Corruption),Year,Overall Rank,Yearly Rank
0,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019,1,1
1,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019,3,2
2,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019,8,3
3,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019,19,4
4,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019,21,5


## Regional Labels

In [13]:
regional = pd.read_html('https://meta.wikimedia.org/wiki/List_of_countries_by_regional_classification')[0]
regional = regional.drop(['Global South'], axis=1)

In [14]:
print('There are {} records for countries and there are {} nulls'.
      format(len(master_df.Country),master_df.Country.isnull().sum()))

There are 782 records for countries and there are 0 nulls


In [15]:
master_df = master_df.merge(regional, on='Country', how='left')
print('There are {} values that we were unable to join'.format(master_df['Region'].isnull().sum()))

There are 71 values that we were unable to join


In [16]:
#These are the countries that we were unable to get a region for
miss_regions = master_df[master_df['Region'].isnull()].groupby('Country').first().reset_index()
miss_regions = miss_regions['Country'].tolist()

## Income Equality

In [17]:
income_equality = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_income_equality')[2]

In [18]:
income_equality.columns = income_equality.columns.droplevel()
income_equality.columns = income_equality.columns.droplevel()
#all I want is the country and the Gini Index (quantified representation of the Lorenz curve, duh!)
income_equality = income_equality.iloc[:,[0,3]]
income_equality.columns = ['Country','Gini Score']

In [19]:
master_df = master_df.merge(income_equality, on='Country', how='left')
print('There are {} values that we were unable to join'.format(master_df['Gini Score'].isnull().sum()))

There are 92 values that we were unable to join


## Suicide Rate

In [20]:
suicide_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_suicide_rate')[3]

suicide_df.columns = suicide_df.columns.droplevel()
suicide_df.columns = ['Both Sexes Rank','Country','Continent',
                      'Suicide Rate','Male Rank','Male Rate',
                      'Female Rank','Female Rate','Male:Female Suicide Ratio']
suicide_df = suicide_df.iloc[:,[0,1,3,4,5,6,7,8]]
# or suicide.drop(['Continent'], axis=1)

In [21]:
#wtf we have country names with (more info) next to them! That's not going to work for joins
#only use the characters that are to the left of a character
suicide_df['Country'] = suicide_df['Country'].str.split('(').str[0]
suicide_df['Country'] = suicide_df['Country'].str.split('[').str[0]
#remove any trailing whitespace from a string
suicide_df['Country'] = suicide_df.Country.str.rstrip()

In [22]:
master_df = master_df.merge(suicide_df[['Country','Suicide Rate','Male:Female Suicide Ratio']], on='Country', how='left')
print('There are {} values that we were unable to join'.format(master_df['Suicide Rate'].isnull().sum()))

There are 42 values that we were unable to join


In [23]:
master_df.isnull().sum()

Country                           0
Happiness Score                   0
Economy (GDP per Capita)          0
Social support                    0
Health (Life Expectancy)          0
Freedom                           0
Generosity                        0
Trust (Government Corruption)     1
Year                              0
Overall Rank                      0
Yearly Rank                       0
Region                           71
Gini Score                       92
Suicide Rate                     42
Male:Female Suicide Ratio        42
dtype: int64

In [28]:
master_df.sort_values(by='Male:Female Suicide Ratio', ascending=False).head(10)

,Country,Happiness Score,Economy (GDP per Capita),Social support,Health (Life Expectancy),Freedom,Generosity,Trust (Government Corruption),Year,Overall Rank,Yearly Rank,Region,Gini Score,Suicide Rate,Male:Female Suicide Ratio
132,Ukraine,4.332,0.820000,1.390000,0.739000,0.178000,0.187000,0.010000,2019,632,133,Europe,25.5,18.5,7.34
758,Ukraine,4.096,0.894652,1.394538,0.575904,0.122975,0.270061,0.023029,2017,677,447,Europe,25.5,18.5,7.34
293,Ukraine,4.103,0.793000,1.413000,0.609000,0.163000,0.187000,0.011000,2018,676,294,Europe,25.5,18.5,7.34
422,Ukraine,4.681,0.799070,1.202780,0.673900,0.251230,0.152750,0.029610,2015,544,738,Europe,25.5,18.5,7.34
592,Ukraine,4.324,0.872870,1.014130,0.586280,0.128590,0.203630,0.018290,2016,633,596,Europe,25.5,18.5,7.34
529,Lithuania,5.813,1.269200,1.064110,0.646740,0.189290,0.020250,0.018200,2016,296,533,Europe,37.7,25.7,7.09
367,Lithuania,5.833,1.147230,1.257450,0.731280,0.213420,0.026410,0.010310,2015,287,683,Europe,37.7,25.7,7.09
678,Lithuania,5.902,1.314582,1.473516,0.628950,0.234232,0.010165,0.011866,2017,263,367,Europe,37.7,25.7,7.09
205,Lithuania,5.952,1.197000,1.527000,0.716000,0.350000,0.026000,0.006000,2018,253,206,Europe,37.7,25.7,7.09
41,Lithuania,6.149,1.238000,1.515000,0.818000,0.291000,0.043000,0.042000,2019,204,42,Europe,37.7,25.7,7.09


In [42]:
#create dataframe for only 2019 (1 record for each country)
df = master_df[master_df['Year'] == '2019']

In [43]:
pycred = pygsheets.authorize(service_file='credentials2.json')
#opening the gsheet and sheet you want to work with
ss = pycred.open('Happiness Data')[0]
#overwrite what is in the sheet with your df
ss.set_dataframe(df,(1,1))

In [44]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 10, random_state = 23)
# Train the model on training data
#rf.fit(train_features, train_labels);